In [52]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.feature_selection import f_classif,SelectKBest,chi2
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
import matplotlib.pyplot as plt

In [53]:
firewall = pd.read_csv("firewall.csv")

C:\Users\Aditya Sai\AppData\Local\Temp\ipykernel_14660\180231136.py:1: DtypeWarning: Columns (49,58,59,60,62,63,64,65,66,67,69,70,72) have mixed types. Specify dtype option on import or set low_memory=False.
  firewall = pd.read_csv("firewall.csv")


In [54]:
firewall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380837 entries, 0 to 380836
Data columns (total 74 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   logver            380837 non-null  int64  
 1   timestamp         380837 non-null  int64  
 2   tz                380837 non-null  object 
 3   devname           380837 non-null  object 
 4   devid             380837 non-null  object 
 5   vd                380837 non-null  object 
 6   date              380837 non-null  object 
 7   time              380837 non-null  object 
 8   logid             380837 non-null  object 
 9   type              380837 non-null  object 
 10  subtype           380837 non-null  object 
 11  level             380837 non-null  object 
 12  eventtime         380837 non-null  int64  
 13  srcip             378837 non-null  object 
 14  srcport           284987 non-null  float64
 15  srcintf           378837 non-null  object 
 16  srcintfrole       37

In [55]:
df = firewall.copy()

In [56]:
numerical_features = ['srcport', 'dstport', 'sessionid', 'proto', 'policyid', 'duration', 'sentbyte', 'rcvdbyte', 'sentpkt', 'rcvdpkt', 'wanin', 'wanout', 'lanin', 'lanout', 'sentdelta', 'rcvddelta', 'crscore', 'craction', 'cpu', 'mem', 'totalsession', 'disk', 'setuprate', 'disklograte', 'fazlograte', 'count', 'incidentserialno', 'countips']
df[numerical_features] = df[numerical_features].fillna(df[numerical_features].mean())

In [57]:
categorical_features = ['srcip', 'srcintf', 'srcintfrole', 'dstip', 'dstintf', 'dstintfrole', 'poluuid', 'action', 'policytype', 'dstcountry', 'srccountry', 'trandisp', 'appcat', 'user', 'ui', 'status', 'reason', 'log', 'eventtype', 'severity', 'attack', 'direction', 'profile', 'ref', 'utmaction', 'bandwidth']
df[categorical_features] = df[categorical_features].fillna(method='ffill')

C:\Users\Aditya Sai\AppData\Local\Temp\ipykernel_14660\540514209.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[categorical_features] = df[categorical_features].fillna(method='ffill')


In [58]:
columns_to_drop = ['logdesc', 'hostname', 'msg', 'app', 'cpu', 'mem', 'totalsession', 'disk', 'bandwidth', 'user', 'ui', 'status', 'count', 'reason', 'log', 'eventtype', 'severity', 'attack', 'direction', 'attackid', 'profile', 'ref', 'incidentserialno', 'utmaction', 'countips']
df.drop(columns=columns_to_drop, inplace=True)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380837 entries, 0 to 380836
Data columns (total 49 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   logver       380837 non-null  int64  
 1   timestamp    380837 non-null  int64  
 2   tz           380837 non-null  object 
 3   devname      380837 non-null  object 
 4   devid        380837 non-null  object 
 5   vd           380837 non-null  object 
 6   date         380837 non-null  object 
 7   time         380837 non-null  object 
 8   logid        380837 non-null  object 
 9   type         380837 non-null  object 
 10  subtype      380837 non-null  object 
 11  level        380837 non-null  object 
 12  eventtime    380837 non-null  int64  
 13  srcip        380837 non-null  object 
 14  srcport      380837 non-null  float64
 15  srcintf      380837 non-null  object 
 16  srcintfrole  380837 non-null  object 
 17  dstip        380837 non-null  object 
 18  dstport      380837 non-

In [60]:
df[["service","crlevel"]] = df[["service","crlevel"]].fillna(method='ffill')

C:\Users\Aditya Sai\AppData\Local\Temp\ipykernel_14660\2661051423.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[["service","crlevel"]] = df[["service","crlevel"]].fillna(method='ffill')


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380837 entries, 0 to 380836
Data columns (total 49 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   logver       380837 non-null  int64  
 1   timestamp    380837 non-null  int64  
 2   tz           380837 non-null  object 
 3   devname      380837 non-null  object 
 4   devid        380837 non-null  object 
 5   vd           380837 non-null  object 
 6   date         380837 non-null  object 
 7   time         380837 non-null  object 
 8   logid        380837 non-null  object 
 9   type         380837 non-null  object 
 10  subtype      380837 non-null  object 
 11  level        380837 non-null  object 
 12  eventtime    380837 non-null  int64  
 13  srcip        380837 non-null  object 
 14  srcport      380837 non-null  float64
 15  srcintf      380837 non-null  object 
 16  srcintfrole  380837 non-null  object 
 17  dstip        380837 non-null  object 
 18  dstport      380837 non-

In [62]:
df["crlevel"] = df["crlevel"].fillna(method='ffill')

C:\Users\Aditya Sai\AppData\Local\Temp\ipykernel_14660\3035383502.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["crlevel"] = df["crlevel"].fillna(method='ffill')


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380837 entries, 0 to 380836
Data columns (total 49 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   logver       380837 non-null  int64  
 1   timestamp    380837 non-null  int64  
 2   tz           380837 non-null  object 
 3   devname      380837 non-null  object 
 4   devid        380837 non-null  object 
 5   vd           380837 non-null  object 
 6   date         380837 non-null  object 
 7   time         380837 non-null  object 
 8   logid        380837 non-null  object 
 9   type         380837 non-null  object 
 10  subtype      380837 non-null  object 
 11  level        380837 non-null  object 
 12  eventtime    380837 non-null  int64  
 13  srcip        380837 non-null  object 
 14  srcport      380837 non-null  float64
 15  srcintf      380837 non-null  object 
 16  srcintfrole  380837 non-null  object 
 17  dstip        380837 non-null  object 
 18  dstport      380837 non-

In [64]:
# Scoring criteria
criteria = {
    'duration': {'safe': 600, 'moderately_safe': 1800, 'unsafe': 3600},
    'sentbyte': {'safe': 1e8, 'moderately_safe': 5e7, 'unsafe': 1e7},
    'crscore': {'safe': 80, 'moderately_safe': 60, 'unsafe': 0},
}


In [65]:
def categorize(row):
    max_category = 'unknown'
    max_threshold = 0
    for feature, thresholds in criteria.items():
        if pd.notna(row[feature]):  # Skip NaN values
            for category, threshold in thresholds.items():
                if row[feature] >= threshold and threshold > max_threshold:
                    max_threshold = threshold
                    max_category = category     
    return max_category

In [66]:
df['category'] = df.apply(categorize, axis=1)

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380837 entries, 0 to 380836
Data columns (total 50 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   logver       380837 non-null  int64  
 1   timestamp    380837 non-null  int64  
 2   tz           380837 non-null  object 
 3   devname      380837 non-null  object 
 4   devid        380837 non-null  object 
 5   vd           380837 non-null  object 
 6   date         380837 non-null  object 
 7   time         380837 non-null  object 
 8   logid        380837 non-null  object 
 9   type         380837 non-null  object 
 10  subtype      380837 non-null  object 
 11  level        380837 non-null  object 
 12  eventtime    380837 non-null  int64  
 13  srcip        380837 non-null  object 
 14  srcport      380837 non-null  float64
 15  srcintf      380837 non-null  object 
 16  srcintfrole  380837 non-null  object 
 17  dstip        380837 non-null  object 
 18  dstport      380837 non-

In [68]:
df.head()

,logver,timestamp,tz,devname,devid,vd,date,time,logid,type,...,lanout,sentdelta,rcvddelta,crscore,craction,crlevel,setuprate,disklograte,fazlograte,category
0,506141727,1698690294,"""UTC+5:30""","""FGT3600C_HA""","""FG3K6C3A15800081""","""root""",2023-10-30,23:54:54,"""0000000013""","""traffic""",...,1067.812715,1721.717462,87030.892007,20.254005,182180.75006,NaN,18.145161,0.0,14.91129,unknown
1,506141727,1698690294,"""UTC+5:30""","""FGT3600C_HA""","""FG3K6C3A15800081""","""root""",2023-10-30,23:54:54,"""0000000013""","""traffic""",...,1067.812715,1721.717462,87030.892007,20.254005,182180.75006,NaN,18.145161,0.0,14.91129,unknown
2,506141727,1698690294,"""UTC+5:30""","""FGT3600C_HA""","""FG3K6C3A15800081""","""root""",2023-10-30,23:54:54,"""0000000013""","""traffic""",...,1148.000000,1721.717462,87030.892007,20.254005,182180.75006,NaN,18.145161,0.0,14.91129,unknown
3,506141727,1698690294,"""UTC+5:30""","""FGT3600C_HA""","""FG3K6C3A15800081""","""root""",2023-10-30,23:54:54,"""0000000013""","""traffic""",...,1067.812715,1721.717462,87030.892007,20.254005,182180.75006,NaN,18.145161,0.0,14.91129,unknown
4,506141727,1698690295,"""UTC+5:30""","""FGT3600C_HA""","""FG3K6C3A15800081""","""root""",2023-10-30,23:54:55,"""0000000013""","""traffic""",...,1148.000000,1721.717462,87030.892007,20.254005,182180.75006,NaN,18.145161,0.0,14.91129,unknown


In [69]:
df["category"].value_counts()

category
unknown            379360
safe                  744
unsafe                383
moderately_safe       350
Name: count, dtype: int64

In [70]:
ob = []
fl = []
num = []
for i in df.columns:
    if df[i].dtype == "object":
        ob.append(i)
for i in df.columns:
    if df[i].dtype == "int64":
        num.append(i)
for i in df.columns:
    if df[i].dtype == "float64":
        fl.append(i)
print(fl)
print(num)
print(ob)

['srcport', 'dstport', 'sessionid', 'proto', 'policyid', 'duration', 'sentbyte', 'rcvdbyte', 'sentpkt', 'rcvdpkt', 'wanin', 'wanout', 'lanin', 'lanout', 'sentdelta', 'rcvddelta', 'crscore', 'craction', 'setuprate', 'disklograte', 'fazlograte']
['logver', 'timestamp', 'eventtime']
['tz', 'devname', 'devid', 'vd', 'date', 'time', 'logid', 'type', 'subtype', 'level', 'srcip', 'srcintf', 'srcintfrole', 'dstip', 'dstintf', 'dstintfrole', 'poluuid', 'action', 'policytype', 'service', 'dstcountry', 'srccountry', 'trandisp', 'appcat', 'crlevel', 'category']


In [71]:
lab = LabelEncoder()
mod_data = df.copy()
for i in ob:
    mod_data[i] = lab.fit_transform(mod_data[i])

In [72]:
mod_data.head()

,logver,timestamp,tz,devname,devid,vd,date,time,logid,type,...,lanout,sentdelta,rcvddelta,crscore,craction,crlevel,setuprate,disklograte,fazlograte,category
0,506141727,1698690294,0,0,1,0,0,18376,1,1,...,1067.812715,1721.717462,87030.892007,20.254005,182180.75006,3,18.145161,0.0,14.91129,2
1,506141727,1698690294,0,0,1,0,0,18376,1,1,...,1067.812715,1721.717462,87030.892007,20.254005,182180.75006,3,18.145161,0.0,14.91129,2
2,506141727,1698690294,0,0,1,0,0,18376,1,1,...,1148.000000,1721.717462,87030.892007,20.254005,182180.75006,3,18.145161,0.0,14.91129,2
3,506141727,1698690294,0,0,1,0,0,18376,1,1,...,1067.812715,1721.717462,87030.892007,20.254005,182180.75006,3,18.145161,0.0,14.91129,2
4,506141727,1698690295,0,0,1,0,0,18377,1,1,...,1148.000000,1721.717462,87030.892007,20.254005,182180.75006,3,18.145161,0.0,14.91129,2


In [73]:
print(mod_data['category'].unique())

[2 1 3 0]


In [74]:
features = []
for i in mod_data.columns:
    if i != "category":
        features.append(i)
X = mod_data[features]
y = mod_data["category"]

In [75]:
mod_data.corr()

,logver,timestamp,tz,devname,devid,vd,date,time,logid,type,...,lanout,sentdelta,rcvddelta,crscore,craction,crlevel,setuprate,disklograte,fazlograte,category
logver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
timestamp,NaN,1.000000,NaN,NaN,-3.005293e-03,NaN,0.255622,0.809355,2.281205e-02,-2.344017e-02,...,-2.783008e-02,2.796096e-03,2.705315e-03,8.523314e-02,-8.521532e-02,-0.208425,-3.551997e-03,NaN,-3.161449e-03,-1.310875e-02
tz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
devname,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
devid,NaN,-0.003005,NaN,NaN,1.000000e+00,NaN,-0.001679,-0.001879,-1.956723e-01,1.904222e-01,...,-3.326002e-14,3.062454e-15,2.982215e-15,1.100915e-13,-1.101315e-13,0.002931,5.206807e-01,NaN,5.802594e-01,-4.738390e-04
vd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date,NaN,0.255622,NaN,NaN,-1.679253e-03,NaN,1.000000,-0.360918,5.190150e-03,-7.892096e-03,...,-6.475654e-03,1.808153e-03,1.742998e-03,-4.599851e-03,4.603945e-03,-0.032994,1.471368e-04,NaN,-5.648371e-04,-6.997182e-03
time,NaN,0.809355,NaN,NaN,-1.879247e-03,NaN,-0.360918,1.000000,1.885643e-02,-1.782156e-02,...,-2.292370e-02,1.599019e-03,1.551023e-03,8.502004e-02,-8.500530e-02,-0.181037,-3.516647e-03,NaN,-2.706800e-03,-8.400965e-03
logid,NaN,0.022812,NaN,NaN,-1.956723e-01,NaN,0.005190,0.018856,1.000000e+00,-9.172189e-01,...,-2.115596e-15,4.577895e-03,3.757604e-03,1.485036e-02,-1.341247e-02,-0.012067,1.204166e-14,NaN,2.615422e-15,-6.535229e-02
type,NaN,-0.023440,NaN,NaN,1.904222e-01,NaN,-0.007892,-0.017822,-9.172189e-01,1.000000e+00,...,8.283481e-16,-2.051533e-16,-1.053110e-16,9.810382e-04,8.066279e-15,0.008102,-1.207026e-14,NaN,-2.605042e-15,-2.480868e-03


In [76]:
sel = SelectKBest(score_func = f_classif,k = 12)
ch = SelectKBest(score_func = chi2,k=12)
data = sel.fit_transform(X,y)
ch_data = ch.fit_transform(X,y)
mod_features = sel.get_support()
ch_features = ch.get_support()
f_cl = X.columns[mod_features]
chi_cl = X.columns[ch_features]

c:\Users\Aditya Sai\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 2  3  5 26 30 35 47] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\Aditya Sai\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [77]:
smote = SMOTE(random_state=42)
X_train_f,X_test_f,y_train_f,y_test_f = train_test_split(X[f_cl],y) #Features found using f score
X_train_chi,X_test_chi,y_train_chi,y_test_chi = train_test_split(X[chi_cl],y) #Features found using chi2 scores
X_resampled_chi, y_resampled_chi = smote.fit_resample(X_train_chi, y_train_chi) #Resampling for f score based features
X_resampled_f, y_resampled_f = smote.fit_resample(X_train_f, y_train_f) #Resampling for chi square based features

In [78]:
# log = LogisticRegression()
# log.fit(X_resampled_chi,y_resampled_chi)

In [79]:
# print(classification_report(log.predict(X_test_chi),y_test_chi))

In [80]:
# dec = DecisionTreeClassifier()
# dec.fit(X_resampled_chi,y_resampled_chi)

In [81]:
# print(classification_report(y_test_chi,dec.predict(X_test_chi)))

In [82]:
# ada = AdaBoostClassifier()
# ada.fit(X_resampled_chi,y_resampled_chi)

In [83]:
# print(classification_report(y_test_chi,ada.predict(X_test_chi)))

In [84]:
ran = RandomForestClassifier()
ran.fit(X_resampled_f,y_resampled_f)

RandomForestClassifier()

In [85]:
print(classification_report(y_test_f,ran.predict(X_test_f)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        88
           1       1.00      1.00      1.00       167
           2       1.00      1.00      1.00     94853
           3       1.00      1.00      1.00       102

    accuracy                           1.00     95210
   macro avg       1.00      1.00      1.00     95210
weighted avg       1.00      1.00      1.00     95210



In [86]:
pc = PCA(n_components = 12)
feats = pc.fit_transform(X)

In [87]:
feature_contributions = pd.DataFrame(pc.components_, columns=X.columns)
feature_contributions

,logver,timestamp,tz,devname,devid,vd,date,time,logid,type,...,lanin,lanout,sentdelta,rcvddelta,crscore,craction,crlevel,setuprate,disklograte,fazlograte
0,-5.044699e-21,0.000035,-3.839242e-28,-3.153759e-36,-2.709384e-17,7.515789e-39,4.897503e-10,0.000026,2.650869e-09,1.112021e-15,...,-3.202420e-06,-3.202420e-06,0.006502,0.374439,-1.583981e-10,8.406763e-07,7.759425e-10,-1.023829e-15,-0.0,-6.463811e-16
1,-2.511834e-17,-0.000054,1.138421e-25,-2.674080e-34,1.798107e-16,5.553365e-36,-6.144137e-11,-0.000053,3.750674e-09,-4.252386e-15,...,4.455871e-06,4.455871e-06,0.012607,0.726690,-3.777899e-10,1.984804e-06,2.862158e-09,7.503602e-15,0.0,4.745544e-15
2,-7.187818e-20,-0.017640,-6.801408e-23,-2.686675e-30,1.561328e-13,1.183634e-32,-1.762020e-07,-0.014331,4.889132e-08,-2.230109e-11,...,1.970980e-05,1.970980e-05,-0.000143,-0.007483,-6.788597e-07,3.558091e-03,2.482901e-07,6.057653e-12,0.0,3.823446e-12
3,1.721219e-17,-0.001286,1.734761e-18,3.130142e-25,1.164620e-13,-1.478888e-27,-1.908979e-08,-0.000928,-1.490930e-07,1.212865e-12,...,-2.676187e-07,-2.676187e-07,0.010043,0.575625,-6.907470e-08,3.610668e-04,1.369579e-09,4.448935e-12,0.0,2.815404e-12
4,1.307319e-18,-0.010709,7.630828e-19,-2.373290e-18,2.653153e-11,7.546215e-21,-1.102607e-07,-0.008629,-3.719478e-07,1.458334e-09,...,1.162668e-05,1.162668e-05,0.000240,0.003476,4.452123e-05,-2.334834e-01,-1.166436e-06,1.447747e-09,0.0,9.502130e-10
5,-3.514147e-19,0.019619,4.064260e-19,4.760152e-19,-7.448864e-11,6.163045e-21,-3.116058e-07,0.025418,6.029125e-07,-3.280619e-09,...,2.427497e-05,2.427497e-05,-0.000080,-0.000654,1.835304e-04,-9.626473e-01,-7.199813e-06,-2.697771e-09,0.0,-1.648176e-09
6,-1.609145e-17,0.476013,1.808388e-18,1.427760e-18,-7.456862e-09,-4.404919e-18,-5.127393e-06,0.571252,4.415248e-06,-3.226973e-07,...,7.318562e-06,7.318562e-06,0.000115,0.001355,-6.867169e-06,3.610249e-02,-6.314231e-06,-3.002545e-07,0.0,-1.871996e-07
7,5.186953e-18,0.018455,-2.248194e-19,2.281293e-20,-8.559853e-10,7.077361e-18,3.898033e-06,-0.054351,-8.179654e-07,-1.573001e-08,...,1.537217e-04,1.537217e-04,-0.000247,-0.000341,-2.474087e-05,1.300573e-01,6.706317e-06,6.051209e-08,-0.0,4.262234e-08
8,-1.338434e-17,-0.091846,-2.603223e-18,-2.504637e-20,2.829465e-09,5.881694e-19,-2.160317e-05,0.311720,1.125158e-06,1.431833e-08,...,-6.970060e-04,-6.970060e-04,-0.001143,-0.000167,-4.465089e-06,2.355868e-02,2.809173e-06,-3.109450e-07,0.0,-2.031001e-07
9,4.807471e-18,0.272037,2.485138e-19,-7.161815e-19,-4.014348e-11,-1.294026e-18,5.477917e-05,-0.751428,3.084215e-06,8.525728e-08,...,1.659134e-03,1.659134e-03,0.001779,0.000296,3.889443e-07,-2.010263e-03,-1.766741e-06,1.184059e-07,0.0,5.646490e-09


In [88]:
most_important_features = feature_contributions.idxmax(axis=1)

In [89]:
pca_analyzed = []
for i, feature in enumerate(most_important_features):
    pca_analyzed.append(feature)
pca_analyzed

['rcvdbyte',
 'rcvddelta',
 'sentbyte',
 'rcvddelta',
 'srcport',
 'dstport',
 'time',
 'dstport',
 'dstport',
 'sentbyte',
 'sentbyte',
 'wanout']

In [90]:
smote = SMOTE(random_state=42)
X_train_pc,X_test_pc,y_train_pc,y_test_pc = train_test_split(X[pca_analyzed],y) #Features found using PCA Analysis
X_resampled_pc, y_resampled_pc = smote.fit_resample(X_train_pc, y_train_pc)

In [91]:
# log.fit(X_resampled_pc,y_resampled_pc)

In [92]:
# print(classification_report(y_test_pc,log.predict(X_test_pc)))

In [93]:
# ran.fit(X_resampled_pc,y_resampled_pc)

In [94]:
# import pickle

# # Specify the path to save the model
# model_path = 'random_forest_model.pkl'

# # Save the model to the specified path
# with open(model_path, 'wb') as file:
#     pickle.dump(ran, file)


In [95]:
import pickle

# Specify the path to the .pkl file
model_path = 'random_forest_model.pkl'

# Load the model from the .pkl file
with open(model_path, 'rb') as file:
    ran = pickle.load(file)


In [96]:
print(classification_report(y_test_pc,ran.predict(X_test_pc)))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        83
           1       0.88      0.94      0.91       193
           2       1.00      1.00      1.00     94839
           3       0.98      0.96      0.97        95

    accuracy                           1.00     95210
   macro avg       0.97      0.97      0.97     95210
weighted avg       1.00      1.00      1.00     95210



In [97]:
# dec.fit(X_resampled_pc,y_resampled_pc)

In [98]:
# print(classification_report(y_test_pc,dec.predict(X_test_pc)))

In [99]:
print(X_resampled_pc.shape)
print(y_resampled_pc.shape)

(1138084, 12)
(1138084,)


In [100]:
y_resampled_pc_onehot = to_categorical(y_resampled_pc, num_classes=4)
X_train, X_test, y_train, y_test = train_test_split(X_resampled_pc, y_resampled_pc_onehot, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# model = Sequential()
# model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(4, activation='softmax'))  # Use softmax for 4 classes
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


22762/22762 [==============================] - 73s 3ms/step - loss: 0.1192 - accuracy: 0.9106 - val_loss: 0.0704 - val_accuracy: 0.9565
Epoch 2/10
22762/22762 [==============================] - 73s 3ms/step - loss: 0.0595 - accuracy: 0.9666 - val_loss: 0.0527 - val_accuracy: 0.9679
Epoch 3/10
22762/22762 [==============================] - 71s 3ms/step - loss: 0.0493 - accuracy: 0.9729 - val_loss: 0.0500 - val_accuracy: 0.9706
Epoch 4/10
22762/22762 [==============================] - 71s 3ms/step - loss: 0.0448 - accuracy: 0.9758 - val_loss: 0.0413 - val_accuracy: 0.9788
Epoch 5/10
22762/22762 [==============================] - 66s 3ms/step - loss: 0.0421 - accuracy: 0.9773 - val_loss: 0.0435 - val_accuracy: 0.9769
Epoch 6/10
22762/22762 [==============================] - 33s 1ms/step - loss: 0.0402 - accuracy: 0.9783 - val_loss: 0.0442 - val_accuracy: 0.9761
Epoch 7/10
22762/22762 [==============================] - 31s 1ms/step - loss: 0.0386 - accuracy: 0.9791 - val_loss:

In [101]:
import pickle

# Specify the path to save the model
model_path = 'neural_network_model.pkl'

# Save the model to the specified path
with open(model_path, 'wb') as file:
    pickle.dump(ran, file)


In [102]:
# history = model.fit(X_train, y_train, epochs=9, batch_size=32, validation_split=0.2)

In [103]:
import pickle
# Specify the path to the .pkl file
file_path = 'neural_network_model.pkl'

# Load the model from the .pkl file
with open(file_path, 'rb') as file:
    model = pickle.load(file)

In [104]:
# y_pred = model.predict(X_test)
# history = model.history.history
# plt.plot(history['loss'], label='Training Loss')
# plt.plot(history['val_loss'], label='Validation Loss')
# plt.title('Training and Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [105]:
# plt.plot(history['accuracy'])
# plt.plot(history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

In [106]:
import warnings
warnings.filterwarnings('ignore')


In [107]:
unique_categories = df["category"].unique().tolist()
print(unique_categories)

['unknown', 'safe', 'unsafe', 'moderately_safe']


In [118]:
import numpy as np

# Select a random row index from X_test
random_index = np.random.randint(0, X_test.shape[0])

# Select the random row from X_test
random_row = X_test[random_index]

In [119]:
print(random_index)

212311


In [123]:
import numpy as np
all_predictions = model.predict(X_test)
unique_predictions = np.unique(all_predictions)
print(unique_predictions)

[2]


In [120]:
# Preprocess the random row
preprocessed_row = scaler.transform(random_row.reshape(1, -1))

# Make a prediction
prediction = model.predict(preprocessed_row)
class_label = np.argmax(prediction)
predicted_class = unique_categories[class_label]
print(f'The predicted class for the selected row using NN is: {predicted_class}')


The predicted class for the selected row using NN is: unknown


In [111]:

# Reshape the random row to match the input shape of the model
random_row_shaped = random_row.reshape(1, -1)

# Make a prediction
prediction = ran.predict(random_row_shaped)
predicted_class = unique_categories[prediction[0]]
print(f'The predicted class for the selected row using random forest is: {predicted_class}')


The predicted class for the selected row using random forest is: unsafe


In [ ]:
num_classes = len(ran.classes_)

print(f'Your Random Forest model can predict {num_classes} classes.')

Your Random Forest model can predict 4 classes.
